<a href="https://colab.research.google.com/github/ririssilvia/Machine-Learning-D4TI/blob/main/js03/js03_Tugas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nama** : Riris Silvia Zahri

**NIM**     : 2241727029 

**Kelas** : TI-4J

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Tugas 2

Buatlah model klasfikasi Multinomial Naive Bayes dengan ketentuan,

1. Menggunakan data `spam.csv`
2. Fitur `TF-IDF` dengan mengaktifkan **stop_words**
3. Evaluasi hasilnya dan bandingkan dengan hasil tugas 1.
4. Berikan kesimpulan fitur mana yang terbaik pada kasus data `spam.csv`

## Load Data

In [5]:
import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_csv('spam.csv')
spam.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


## Building a Bag of Words model
Setelah  memiliki dokumen dalam  representasi kata,  dapat menggunakan vektor tersebut sebagai input untuk model . SpaCy menangani banyak konversi kata dan membangun model linier sederhana untuk Anda dengan kelas `TextCategorizer`.

TextCategorizer adalah pipa SpaCy. Pipa adalah kelas untuk memproses dan mengubah token.

In [1]:
pip install -U spacy==2.3.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Klasifikasi clasess positif dan negative

kata positif atau negatif, dengan seeting `"exclusive_classes"` ke `True`.  mengonfigurasinya dengan arsitektur bag of words (`"bow"`). SpaCy juga menyediakan arsitektur jaringan saraf convolutional, tetapi lebih kompleks daripada yang dibutuhkan saat ini.



In [6]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

## Menambahkan label ke model.
Di sini "ham" untuk pesan sebenarnya, "spam" adalah pesan spam.

In [7]:
# Add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

## Training Model Pengkategori Teks

perlu mengonversi label dalam data ke bentuk yang dibutuhkan TextCategorizer. Untuk setiap contoh, membuat kamus dengan nilai boolean untuk setiap kelas. Dalam hal ini, jika sebuah teks adalah "ham", kita memerlukan kamus `{'ham': True, 'spam': False}`. Model mencari label ini di dalam kamus lain dengan kunci `'cats'`

In [8]:
# Hold out a bit of data for testing
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]

## menggabungkan teks dan label menjadi satu daftar.

In [9]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

## Training classifier. 
 menetapkan beberapa benih acak sehingga  mendapatkan hasil yang dapat diulang. Kemudian kita perlu membuat `optimizer` menggunakan `nlp.begin_training()`. SpaCy menggunakan pengoptimal ini untuk memperbarui model. Secara umum lebih efisien untuk melatih model dalam batch kecil. Untuk ini, SpaCy menyediakan fungsi `minibatch` yang mengembalikan generator yang menghasilkan minibatch untuk pelatihan. Terakhir, minibatch dipecah menjadi teks dan label, kemudian digunakan dengan `nlp.update` untuk memperbarui parameter model.

In [10]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

## Mengacak data pelatihan di awal setiap loop.

In [11]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.43189741921099767}
{'textcat': 0.6474976215331196}
{'textcat': 0.7842154536487618}
{'textcat': 0.8716683716818165}
{'textcat': 0.9280939335008995}
{'textcat': 0.9655779922872296}
{'textcat': 0.9939651840090362}
{'textcat': 1.0127976631523663}
{'textcat': 1.0275637812859075}
{'textcat': 1.0378531470013608}


## Membuat Prediksi
 mencoba membuat prediksi. Cara termudah untuk melakukannya adalah dengan menggunakan metode `predict()` dari pipa TextCategorizer. Pertama, teks input perlu diberi token dengan `nlp.tokenizer`.

In [12]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.9994397e-01 5.6023764e-05]
 [1.1491306e-02 9.8850864e-01]]
